# NLP 

## GEnsim Topic modeling

In [4]:
import pandas as pd
from tqdm import tqdm
import re
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
# np.random.seed(2018)
import pandas as pd 
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\emreb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
stemmer = SnowballStemmer("english")

In [5]:
ratrev_df = pd.read_csv('./dbs/ratingsandreviews.csv')

In [21]:
ratrev_df.dropna(inplace=True)
ratrev_df.reset_index(drop=True)

,name,rating,review,sentiment
0,Jalsa,4.0,A beautiful place to dine inThe interiors take...,2
1,Jalsa,4.0,I was here for dinner with my family on a week...,2
2,Jalsa,2.0,Its a restaurant near to Banashankari BDA Me a...,0
3,Jalsa,4.0,We went here on a weekend and one of us had th...,2
4,Jalsa,5.0,The best thing about the place is its ambiance...,2
...,...,...,...,...
1045909,The Boozy Griffin,4.0,Went to this place for a group outing Loved th...,2
1045910,The Boozy Griffin,5.0,The place is really amazing with a European de...,2
1045911,The Boozy Griffin,5.0,One of my fav pub in Bangalore Such a lively a...,2
1045912,The Boozy Griffin,4.0,Very attractive place with theme setup and dim...,2


In [98]:
data_text = ratrev_df[['review']]
data_text['index'] = data_text.index
documents = data_text

C:\Users\emreb\AppData\Local\Temp\ipykernel_11864\3089262578.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_text['index'] = data_text.index


In [6]:
ratrev_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1045914 entries, 0 to 1048574
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   name    1045914 non-null  object 
 1   rating  1045914 non-null  float64
 2   review  1045914 non-null  object 
dtypes: float64(1), object(2)
memory usage: 31.9+ MB


In [7]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

In [8]:
doc_sample = documents[documents['index'] == 4010].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['Disgusting', 'food', 'and', 'hopeless', 'service', '\nI', 'will', 'come', 'back', 'and', 'write', 'the', 'rest', 'of', 'review', 'when', 'I', 'have', 'time\n\nWill', 'be', 'back', 'just', 'for', 'the', 'review']


 tokenized and lemmatized document: 
['disgust', 'food', 'hopeless', 'servic', 'come', 'write', 'rest', 'review', 'time', 'review']


In [ ]:
processed_docs = documents['review'].map(preprocess)
processed_docs[:10]

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 allth
1 avail
2 beauti
3 betterwould
4 bread
5 christma
6 come
7 complaint
8 compromis
9 dine
10 go


In [ ]:
dictionary.filter_extremes(no_below=5, no_above=0.1, keep_n= 100000)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[5200]

[(72, 3),
 (95, 1),
 (123, 1),
 (193, 1),
 (1328, 1),
 (1970, 1),
 (2538, 1),
 (3274, 1),
 (3634, 1),
 (3876, 1),
 (3896, 1),
 (5016, 1),
 (5017, 1),
 (5018, 1),
 (5019, 1)]

In [ ]:
bow_doc_5200 = bow_corpus[5200]
for i in range(len(bow_doc_5200)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_5200[i][0], 
                                               dictionary[bow_doc_5200[i][0]], 
bow_doc_5200[i][1]))

Word 72 ("qualiti") appears 3 time.
Word 95 ("recommend") appears 1 time.
Word 123 ("dish") appears 1 time.
Word 193 ("foodi") appears 1 time.
Word 1328 ("deserv") appears 1 time.
Word 1970 ("focus") appears 1 time.
Word 2538 ("exceed") appears 1 time.
Word 3274 ("industri") appears 1 time.
Word 3634 ("compon") appears 1 time.
Word 3876 ("pleasur") appears 1 time.
Word 3896 ("frank") appears 1 time.
Word 5016 ("donei") appears 1 time.
Word 5017 ("patronag") appears 1 time.
Word 5018 ("proprietor") appears 1 time.
Word 5019 ("segment") appears 1 time.


In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.29924714890308257),
 (1, 0.13005345907152863),
 (2, 0.13342348737561568),
 (3, 0.4296215895094962),
 (4, 0.13816474185454491),
 (5, 0.22413455741460614),
 (6, 0.08718436252473614),
 (7, 0.19439072472403165),
 (8, 0.21606987036123496),
 (9, 0.17476238440192973),
 (10, 0.14637053524324725),
 (11, 0.3919883247319834),
 (12, 0.10777446945556099),
 (13, 0.13755235964147125),
 (14, 0.16121541854491037),
 (15, 0.27431353780051937),
 (16, 0.19787465668863302),
 (17, 0.37287817903433224),
 (18, 0.11904599696944951)]


In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary,  workers=2,passes=2)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.021*"dont" + 0.020*"money" + 0.016*"worst" + 0.011*"didnt" + 0.011*"experi" + 0.011*"qualiti" + 0.011*"valu" + 0.011*"disappoint" + 0.010*"take" + 0.010*"rat"
Topic: 1 
Words: 0.023*"starter" + 0.019*"buffet" + 0.019*"main" + 0.017*"menu" + 0.016*"cours" + 0.015*"experi" + 0.013*"dish" + 0.012*"lunch" + 0.012*"dessert" + 0.012*"varieti"
Topic: 2 
Words: 0.037*"paneer" + 0.031*"masala" + 0.021*"butter" + 0.019*"roti" + 0.018*"dosa" + 0.017*"paratha" + 0.016*"curri" + 0.014*"tikka" + 0.012*"rice" + 0.012*"thali"
Topic: 3 
Words: 0.056*"pizza" + 0.028*"chocol" + 0.023*"cream" + 0.019*"cake" + 0.018*"dessert" + 0.014*"shake" + 0.013*"tri" + 0.013*"coffe" + 0.010*"cafe" + 0.009*"flavour"
Topic: 4 
Words: 0.026*"chees" + 0.025*"burger" + 0.024*"fri" + 0.018*"pizza" + 0.017*"pasta" + 0.016*"sauc" + 0.012*"mushroom" + 0.011*"peri" + 0.011*"chilli" + 0.011*"grill"
Topic: 5 
Words: 0.030*"indian" + 0.017*"north" + 0.016*"breakfast" + 0.013*"small" + 0.013*"locat" + 0.013*"quic

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.016*"pancak" + 0.014*"paratha" + 0.012*"super" + 0.010*"bengali" + 0.010*"sweet" + 0.008*"aloo" + 0.008*"kachori" + 0.006*"puri" + 0.005*"stale" + 0.005*"chat"
Topic: 1 Word: 0.048*"biryani" + 0.028*"biriyani" + 0.018*"awesom" + 0.015*"spici" + 0.015*"mutton" + 0.014*"rice" + 0.012*"piec" + 0.010*"excel" + 0.009*"curri" + 0.008*"quantiti"
Topic: 2 Word: 0.013*"paneer" + 0.010*"noodl" + 0.009*"butter" + 0.008*"rice" + 0.008*"fri" + 0.007*"chilli" + 0.007*"starter" + 0.007*"soup" + 0.007*"tikka" + 0.006*"main"
Topic: 3 Word: 0.007*"danc" + 0.006*"floor" + 0.006*"drink" + 0.005*"tabl" + 0.005*"music" + 0.005*"come" + 0.004*"experi" + 0.004*"zomato" + 0.004*"manag" + 0.004*"gold"
Topic: 4 Word: 0.020*"pizza" + 0.010*"cake" + 0.008*"burger" + 0.008*"pasta" + 0.008*"dessert" + 0.007*"chees" + 0.006*"delici" + 0.005*"steak" + 0.005*"sauc" + 0.005*"crust"
Topic: 5 Word: 0.011*"roll" + 0.008*"chai" + 0.007*"pakoda" + 0.006*"omelett" + 0.006*"marath" + 0.005*"english" + 0.005*"w

In [ ]:
for index, score in sorted(lda_model[bow_corpus[5200]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.6343553066253662	 
Topic: 0.021*"dont" + 0.020*"money" + 0.016*"worst" + 0.011*"didnt" + 0.011*"experi" + 0.011*"qualiti" + 0.011*"valu" + 0.011*"disappoint" + 0.010*"take" + 0.010*"rat"

Score: 0.1650393307209015	 
Topic: 0.082*"biryani" + 0.036*"biriyani" + 0.032*"quantiti" + 0.027*"qualiti" + 0.022*"mutton" + 0.021*"awesom" + 0.019*"tasti" + 0.019*"spici" + 0.016*"rice" + 0.014*"piec"

Score: 0.15934547781944275	 
Topic: 0.023*"starter" + 0.019*"buffet" + 0.019*"main" + 0.017*"menu" + 0.016*"cours" + 0.015*"experi" + 0.013*"dish" + 0.012*"lunch" + 0.012*"dessert" + 0.012*"varieti"


In [ ]:
for index, score in sorted(lda_model_tfidf[bow_corpus[5200]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.41239386796951294	 
Topic: 0.013*"indian" + 0.012*"tasti" + 0.010*"north" + 0.010*"meal" + 0.009*"dosa" + 0.008*"home" + 0.007*"money" + 0.007*"valu" + 0.007*"thali" + 0.006*"qualiti"

Score: 0.33113551139831543	 
Topic: 0.012*"waffl" + 0.012*"chocol" + 0.011*"cream" + 0.010*"burger" + 0.009*"shake" + 0.007*"chees" + 0.007*"sandwich" + 0.006*"cheesecak" + 0.006*"cake" + 0.006*"coffe"

Score: 0.21521970629692078	 
Topic: 0.007*"danc" + 0.006*"floor" + 0.006*"drink" + 0.005*"tabl" + 0.005*"music" + 0.005*"come" + 0.004*"experi" + 0.004*"zomato" + 0.004*"manag" + 0.004*"gold"


In [ ]:
unseen_document= '''I met a traveller from an antique land,
                    Who said—“Two vast and trunkless legs of stone
                    Stand in the desert. . . . Near them, on the sand,
                    Half sunk a shattered visage lies, whose frown,
                    And wrinkled lip, and sneer of cold command,
                    Tell that its sculptor well those passions read
                    Which yet survive, stamped on these lifeless things,
                    The hand that mocked them, and the heart that fed;
                    And on the pedestal, these words appear:
                    My name is Ozymandias, King of Kings;'''

print(unseen_document)
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

I met a traveller from an antique land,
                    Who said—“Two vast and trunkless legs of stone
                    Stand in the desert. . . . Near them, on the sand,
                    Half sunk a shattered visage lies, whose frown,
                    And wrinkled lip, and sneer of cold command,
                    Tell that its sculptor well those passions read
                    Which yet survive, stamped on these lifeless things,
                    The hand that mocked them, and the heart that fed;
                    And on the pedestal, these words appear:
                    My name is Ozymandias, King of Kings;
Score: 0.4889273941516876	 Topic: 0.021*"dont" + 0.020*"money" + 0.016*"worst" + 0.011*"didnt" + 0.011*"experi"
Score: 0.18737132847309113	 Topic: 0.023*"starter" + 0.019*"buffet" + 0.019*"main" + 0.017*"menu" + 0.016*"cours"
Score: 0.09491205960512161	 Topic: 0.026*"chees" + 0.025*"burger" + 0.024*"fri" + 0.018*"pizza" + 0.017*"pasta"
Score: 0.07232675701

In [ ]:
# from gensim.models import CoherenceModel

In [ ]:
print('\nPerplexity: ', lda_model.log_perplexity(bow_corpus))


Perplexity:  -7.909063692733605


In [ ]:
import pyLDAvis
import pyLDAvis.gensim 

c:\Users\emreb\Documents\projects\translatortelegram\transtele\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [ ]:
pyLDAvis.enable_notebook()
visal = pyLDAvis.gensim.prepare(lda_model,  bow_corpus,dictionary)
visal

c:\Users\emreb\Documents\projects\translatortelegram\transtele\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.177891 -0.068208       1        1  14.401764
1     -0.122755  0.141927       2        1  13.524854
8     -0.147235  0.185528       3        1  12.962277
4      0.187785  0.051817       4        1  11.161997
5     -0.083543  0.092276       5        1  11.047848
3      0.022484  0.101434       6        1  11.042948
2      0.197412 -0.115401       7        1   7.917610
9      0.072489 -0.022018       8        1   7.648699
7      0.249950 -0.045274       9        1   5.546535
6     -0.198697 -0.322081      10        1   4.745467, topic_info=          Term           Freq          Total Category  logprob  loglift
261    biryani  120905.000000  120905.000000  Default  30.0000  30.0000
324      pizza  156167.000000  156167.000000  Default  29.0000  29.0000
1006  deliveri   80364.000000   80364.000000  Default  28.0000  28.0000
89         fri   85535.000000   85535.000000  Default  27.0000  27.0000
418      deliv   51427.000000   51427.000000  Default  26.0000  26.0000
...        ...            ...            ...      ...      ...      ...
29       fresh    6300.721106   48459.977530  Topic10  -4.9758   1.0079
150     custom    5608.619582   34666.237942  Topic10  -5.0922   1.2265
97       tasti    5964.690122   87485.607848  Topic10  -5.0306   0.3624
383      happi    5012.491327   28614.999221  Topic10  -5.2046   1.3060
471     person    5029.729843   37066.933598  Topic10  -5.2011   1.0506

[701 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
733       1  0.117153  absolut
733       2  0.204007  absolut
733       3  0.018852  absolut
733       4  0.211297  absolut
733       5  0.001481  absolut
...     ...       ...      ...
173       4  0.082647   zomato
173       8  0.000472   zomato
173       9  0.000755   zomato
173      10  0.249282   zomato
860       3  0.999742     zone

[1820 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 9, 5, 6, 4, 3, 10, 8, 7])

In [ ]:
visaltf_idf = pyLDAvis.gensim.prepare(lda_model_tfidf,  corpus_tfidf,dictionary)
visaltf_idf

c:\Users\emreb\Documents\projects\translatortelegram\transtele\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.139055  0.055887       1        1  14.163883
3      0.072337  0.098183       2        1  13.205912
4      0.130216 -0.008432       3        1  11.758034
7      0.151350 -0.025619       4        1  11.740604
8     -0.031050 -0.033131       5        1  11.537305
2      0.005120 -0.076434       6        1  11.099009
9     -0.206323  0.187439       7        1   8.091923
1     -0.141233 -0.002408       8        1   7.370980
0     -0.122480 -0.226201       9        1   5.791906
5      0.003007  0.030714      10        1   5.240444, topic_info=           Term          Freq         Total Category  logprob  loglift
261     biryani  18010.000000  18010.000000  Default  30.0000  30.0000
238       worst   9977.000000   9977.000000  Default  29.0000  29.0000
1006   deliveri  13072.000000  13072.000000  Default  28.0000  28.0000
175    biriyani   9144.000000   9144.000000  Default  27.0000  27.0000
418       deliv   9222.000000   9222.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
676   breakfast    648.083127   6051.806459  Topic10  -5.6690   0.7147
1704      jumbo    453.606450    797.222993  Topic10  -6.0258   2.3849
797       onion    493.602305   2991.234171  Topic10  -5.9413   1.1471
56         take    492.245565   6677.531409  Topic10  -5.9440   0.3412
69       experi    468.176586  12829.036261  Topic10  -5.9942  -0.3619

[806 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
11916      9  0.996719      aalo
5008       3  0.997003     aaloo
3730       6  0.075357   account
3730       8  0.923126   account
5858       5  0.992405       act
...      ...       ...       ...
860        5  0.003989      zone
860        6  0.022604      zone
860        7  0.015956      zone
860        8  0.071801      zone
4122      10  0.996931  zucchini

[2859 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 4, 5, 8, 9, 3, 10, 2, 1, 6])

## Sentinement Analysis

In [2]:
from torchtext.vocab import FastText
import pandas as pd
import spacy
import torch 
import torchtext  ## for downloading the data from pytorch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import default_collate
from torch.utils.data.dataset import random_split
from torch.utils.data.dataset import ConcatDataset
from torch.utils.data.dataset import Subset

c:\Users\emreb\Documents\projects\translatortelegram\transtele\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
ratrev_df['sentiment'] = ratrev_df['rating'].apply(lambda x: 2 if x>3.5 else (1 if 3.5>=x>2.5 else 0))

In [70]:
# def clean_text(text):  
#     pat1 = r'@[^ ]+'                   #@signs and value
#     pat2 = r'https?://[A-Za-z0-9./]+'  #links
#     pat3 = r'\'s'                      #floating s's
#     pat4 = r'\#\w+'                     # hashtags and value
#     pat5 = r'&amp '
#     pat6 = r'[^A-Za-z\s]'         #remove non-alphabet
#     combined_pat = r'|'.join((pat1, pat2,pat3,pat4,pat5, pat6))
#     text = re.sub(combined_pat,"",text).lower()
#     return text.strip()

In [71]:
# ratrev_df["cleaned_review"] = ratrev_df["review"].apply(clean_text)

In [23]:
ratrev_df.iloc[4500:4520]

,name,rating,review,sentiment
4510,Anda Ka Funda,2.5,This is one place Id think twice before going\...,0
4511,Anda Ka Funda,4.5,Excellent place for your taste buds If you are...,2
4512,Anda Ka Funda,4.0,Sattu paratha was really good here The place i...,2
4513,Anda Ka Funda,4.5,Good Food Value for money specially in souther...,2
4514,Anda Ka Funda,4.0,I usually dont go to this joint but theyre all...,2
4515,Shri Vinayaka Ice N Juice,4.0,According to me the best chats you will get ne...,2
4516,Shri Vinayaka Ice N Juice,5.0,Not gonna lie the place is quite crowded and b...,2
4517,Shri Vinayaka Ice N Juice,3.0,Good place for chaat items and juice We tried ...,1
4518,Shri Vinayaka Ice N Juice,3.0,Service 35\nTaste 35\nParking 355\n\nI have...,1
4519,Shri Vinayaka Ice N Juice,3.0,There was small cafe when i was young in this ...,1


### pytorch sol

In [8]:
nlp = spacy.load('en_core_web_sm')

In [9]:
def preprocessing(sentences):
    """ perform : tokenization & lemmatization, removes stopwords & punctuations, lower cases"""

    doc = nlp(sentences)
    # tokenization, we return the tokens that aren't stopwords or punctuations
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return tokens

In [ ]:
# mele = [preprocess(i) for i in ratrev_df.review]

In [10]:
sent_df = pd.DataFrame(columns=['Sent','Text'])

In [12]:
# sent_df['Text']= processed_docs

In [13]:
sent_df['Sent']= ratrev_df.sentiment

In [14]:
fasttext = FastText("simple")

In [15]:
fasttext.dim

300

In [16]:
def token_encoder(token, vec):

    """ encodes a single word"""
    if token == "<pad>":
        return 1
    else:
        try:
            return vec.stoi[token]  # if the token is in the vocabulary, string to index
        except:
            if type(token) != str:
                print("Error, we need a word which is in string format")
            else:
                return 0  # if the token is `not in the vocabulary, return 0

In [21]:

fasttext.itos[2610], fasttext.stoi["hello"]

('hello', 2610)

In [17]:
def encoder(tokens, voc):
    """ encodes a list of tokens"""
    return [token_encoder(token, voc) for token in tokens]

In [23]:
encoder(sent_df.Text.iloc[1][:10],fasttext)

[7387, 0, 20599, 0, 0, 0, 197, 197, 7050, 123]

In [24]:
sent_df.Text.iloc[1][:10]

['dinner',
 'famili',
 'weekday',
 'restaur',
 'complet',
 'ambienc',
 'good',
 'good',
 'hindi',
 'music']

In [18]:
def padding(list_of_indexes, max_seq_len, padding_index = 1):
    output = list_of_indexes + (max_seq_len - len(list_of_indexes)) * [padding_index]
    return output[:max_seq_len]

In [19]:
class TrainData(Dataset):
    def __init__(self, df, max_seq_len =32):
        self.max_seq_len = max_seq_len
        train_iter =iter(df.review.values)
        self.vec = FastText("simple")
        self.vec.vectors[1] = -torch.ones(self.vec.vectors[1].shape[0])
        self.vec.vectors[0] = torch.zeros(self.vec.vectors[0].shape[0])
        self.vectorize = lambda x: self.vec.vectors[x]
        self.labels = df.sentiment.values
        self.sequences =[padding(encoder(preprocessing(sequence),fasttext),max_seq_len=32) for sequence in train_iter]

    def __len__(self):
        return len(self.sequences)
           
    def __getitem__(self, idx):
        assert len(self.sequences[idx]) == self.max_seq_len
        return self.sequences[idx], self.labels[idx]  #idx so thatT we dont flood the memory with tensors 

        
   

In [24]:
dataset = TrainData(ratrev_df.sample(20000))

In [25]:
dataset[1]

([261,
  0,
  0,
  335,
  383,
  341,
  2609,
  10324,
  0,
  1185,
  45840,
  2085,
  40,
  261,
  37782,
  0,
  994,
  229,
  506,
  1618,
  1154,
  404,
  404,
  0,
  5160,
  44638,
  2716,
  0,
  5160,
  44638,
  197,
  4385],
 1)

In [27]:
def collate(batch, vectorizer = dataset.vectorize):
    # batch of sentence
    # from these sentences we want the tokens, for each sentence
    # we want to get the tokens, and then we want to get the embeddings
    # we need to pass vectorizer
    inputs = torch.stack([torch.stack([vectorizer(token) for token in sentence[0]]) for sentence in batch])  
    targets = torch.LongTensor([item[1] for item in batch])

    return inputs, targets

In [28]:
batch_size = 16
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate)
train_iter = iter(train_loader)
sentence, target = next(train_iter)
sentence.shape

torch.Size([16, 32, 300])

In [ ]:
datasettest = 

In [36]:
test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate)
test_iter = iter(test_loader)
t_sent, t_target = next(test_iter)
t_sent.shape

torch.Size([16, 32, 300])

In [29]:
emb_dim = fasttext.dim 

from torch import nn
import torch.nn.functional as F
emb_dim = 300
class Classifier(nn.Module):
    def __init__(self, max_seq_len, emb_dim, hidden1=16, hidden2=16):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(max_seq_len*emb_dim, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, 5)
        self.out = nn.LogSoftmax(dim=1)
    
    
    def forward(self, inputs):
        x = F.relu(self.fc1(inputs.squeeze(1).float()))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return self.out(x)

In [ ]:
no_layers = 2
vocab_size = len(vocab) + 1 #extra 1 for padding
embedding_dim = 64
output_dim = 1
hidden_dim = 256

In [ ]:
class SentimentRNN(nn.Module):
    def __init__(self,no_layers,vocab_size,hidden_dim,embedding_dim,drop_prob=0.5):
        super(SentimentRNN,self).__init__()
 
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
 
        self.no_layers = no_layers
        self.vocab_size = vocab_size
    
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #lstm
        self.lstm = nn.LSTM(input_size=embedding_dim,hidden_size=self.hidden_dim,
                           num_layers=no_layers, batch_first=True)
        
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
    
        # linear and sigmoid layer
        self.fc = nn.Linear(self.hidden_dim, output_dim)
        self.sig = nn.Sigmoid()
        
    def forward(self,x,hidden):
        batch_size = x.size(0)
        # embeddings and lstm_out
        embeds = self.embedding(x)  # shape: B x S x Feature   since batch = True
        #print(embeds.shape)  #[50, 500, 1000]
        lstm_out, hidden = self.lstm(embeds, hidden)
        
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim) 
        
        # dropout and fully connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)

        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
        
        
        
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        h0 = torch.zeros((self.no_layers,batch_size,self.hidden_dim)).to(device)
        c0 = torch.zeros((self.no_layers,batch_size,self.hidden_dim)).to(device)
        hidden = (h0,c0)
        return hidden

In [30]:
MAX_SEQ_LEN = 32
model = Classifier(MAX_SEQ_LEN, 300, 16, 16)
model

Classifier(
  (fc1): Linear(in_features=9600, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=5, bias=True)
  (out): LogSoftmax(dim=1)
)

In [31]:
criterion1 = nn.CrossEntropyLoss()

In [32]:
criterion = nn.NLLLoss()
optimizer = optim.Adagrad(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

In [34]:
epochs = 20
print_every = 40

for e in range(epochs):
    running_loss = 0
    print(f"Epoch: {e+1}/{epochs}")

    for i, (sentences, labels) in enumerate(iter(train_loader)):

        sentences.resize_(sentences.size()[0], 32* emb_dim)
        
        optimizer.zero_grad()
        
        output = model.forward(sentences)   # 1) Forward pass
        loss = criterion(output, labels) # 2) Compute loss
        loss.backward()                  # 3) Backward pass
        optimizer.step()                 # 4) Update model
        
        running_loss += loss.item()
        
        if i % print_every == 0:
            print(f"\tIteration: {i}\t Loss: {running_loss/print_every:.4f}")
            running_loss = 0

Epoch: 1/20
	Iteration: 0	 Loss: 0.0161
	Iteration: 40	 Loss: 0.6482
	Iteration: 80	 Loss: 0.6014
	Iteration: 120	 Loss: 0.6311
	Iteration: 160	 Loss: 0.6201
	Iteration: 200	 Loss: 0.6086
	Iteration: 240	 Loss: 0.6318
	Iteration: 280	 Loss: 0.6331
	Iteration: 320	 Loss: 0.5906
	Iteration: 360	 Loss: 0.5921
	Iteration: 400	 Loss: 0.6041
	Iteration: 440	 Loss: 0.6176
	Iteration: 480	 Loss: 0.5890
	Iteration: 520	 Loss: 0.6181
	Iteration: 560	 Loss: 0.6274
	Iteration: 600	 Loss: 0.6137
	Iteration: 640	 Loss: 0.6478
	Iteration: 680	 Loss: 0.6142
	Iteration: 720	 Loss: 0.5946
	Iteration: 760	 Loss: 0.6070
	Iteration: 800	 Loss: 0.6101
	Iteration: 840	 Loss: 0.5963
	Iteration: 880	 Loss: 0.6190
	Iteration: 920	 Loss: 0.6445
	Iteration: 960	 Loss: 0.6466
	Iteration: 1000	 Loss: 0.5976
	Iteration: 1040	 Loss: 0.5998
	Iteration: 1080	 Loss: 0.6642
	Iteration: 1120	 Loss: 0.6319
	Iteration: 1160	 Loss: 0.6456
	Iteration: 1200	 Loss: 0.6450
	Iteration: 1240	 Loss: 0.6405
Epoch: 2/20
	Iteration: 0